# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [9]:
# Write your code below.
%load_ext dotenv
%dotenv 
# pip install python-dotenv
#PRICE_DATA='c:\Users\yihan\miniconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\_pyarrow_compat.py:17'





The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv
cannot find .env file


In [10]:
from dotenv import load_dotenv
import os

dotenv_path = r'C:\Users\yihan\production\05_src\.env'

# Load environment variables from the .env file
load_dotenv(dotenv_path)

# Access the PRICE_DATA variable
price_data_path = os.getenv('PRICE_DATA')

print(f"PRICE_DATA is set to: {price_data_path}")

PRICE_DATA is set to: ../05_src/data/prices/


In [11]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [12]:
import os
from glob import glob

# Load the environment variable PRICE_DATA
price_data_path = os.getenv('PRICE_DATA')

# Use glob to find all parquet files in the directory
parquet_files = glob(os.path.join(price_data_path, '*.parquet'))





For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [13]:
# Write your code below.

# Load your data (adjust the path to where your data is stored)
ddf = dd.read_parquet(r'C:\Users\yihan\production\05_src\data\features\stock_features.parquet')

# Repartition the DataFrame to ensure larger partitions
ddf = ddf.repartition(partition_size='100MB')

# Add lags for Close and Adj Close
ddf['Close_lag'] = ddf['Close'].shift(1)
ddf['Adj Close_lag'] = ddf['Adj Close'].shift(1)

# Add returns based on Adjusted Close
ddf['returns'] = (ddf['Adj Close'] / ddf['Adj Close_lag']) - 1

# Add the high-low range for the day
ddf['hi_lo_range'] = ddf['High'] - ddf['Low']

# Assign the result to dd_feat
dd_feat = ddf.persist()

# Print the first few rows to check the result
print(dd_feat.head())



             Date    Open   High     Low   Close  Adj Close   Volume  \
ticker                                                                 
HUM    2000-01-03  8.3125  8.375  7.3750  7.5625   6.752523  1287300   
HUM    2000-01-04  7.3750  7.875  7.3750  7.6250   6.808330  1238300   
HUM    2000-01-05  7.5000  7.875  7.5000  7.8125   6.975746  1096300   
HUM    2000-01-06  7.7500  8.250  7.5000  8.1250   7.254777  1026700   
HUM    2000-01-07  8.1875  9.125  8.0625  8.7500   7.812839  2419300   

             sector            subsector  year  Close_lag_1   returns  \
ticker                                                                  
HUM     Health Care  Managed Health Care  2000      10.0000       NaN   
HUM     Health Care  Managed Health Care  2000       7.5625  0.008265   
HUM     Health Care  Managed Health Care  2000       7.6250  0.024590   
HUM     Health Care  Managed Health Care  2000       7.8125  0.040000   
HUM     Health Care  Managed Health Care  2000       8.12

+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [14]:
# Write your code below.

# Now convert the Dask DataFrame to a Pandas DataFrame
pandas_df = dd_feat.compute()

# Add a rolling average return calculation with a window of 10 days
pandas_df['rolling_avg_return'] = pandas_df['returns'].rolling(window=10).mean()

# Display the first few rows
print(pandas_df.head())











             Date    Open   High     Low   Close  Adj Close   Volume  \
ticker                                                                 
HUM    2000-01-03  8.3125  8.375  7.3750  7.5625   6.752523  1287300   
HUM    2000-01-04  7.3750  7.875  7.3750  7.6250   6.808330  1238300   
HUM    2000-01-05  7.5000  7.875  7.5000  7.8125   6.975746  1096300   
HUM    2000-01-06  7.7500  8.250  7.5000  8.1250   7.254777  1026700   
HUM    2000-01-07  8.1875  9.125  8.0625  8.7500   7.812839  2419300   

             sector            subsector  year  Close_lag_1   returns  \
ticker                                                                  
HUM     Health Care  Managed Health Care  2000      10.0000       NaN   
HUM     Health Care  Managed Health Care  2000       7.5625  0.008265   
HUM     Health Care  Managed Health Care  2000       7.6250  0.024590   
HUM     Health Care  Managed Health Care  2000       7.8125  0.040000   
HUM     Health Care  Managed Health Care  2000       8.12

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?

No, it is not necessary to convert to Pandas to calculate the moving average return. Dask can perform rolling computations on Dask DataFrames.


+ Would it have been better to do it in Dask? Why?

Yes, it would have been better to perform the rolling average calculation in Dask. Dask is built to efficiently manage datasets that exceed memory limits and can process data concurrently, making it more effective for larger datasets. Converting to Pandas may result in memory-related challenges if the dataset is substantial, and it diminishes the advantages of Dask’s lazy evaluation and parallel processing features.

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.